* use counter in instruction definition to keep track how number of lines used and supposed location in ROM.mem file
* optional label to help with jumping -> replace with line number at later stage

In [52]:
import os
import numpy as np
from enum import Enum

In [2]:
# Data dir
data_dir = '../data'

### ROM data generation

In [3]:
ROM_ADDR_WIDTH = 8
ADDR_COUNT = 2 ** ROM_ADDR_WIDTH
ROM_FILE = os.path.join(data_dir, 'ROM_test.mem')
data = '\n'.join([f'{i:02X}' for i in range(ADDR_COUNT)])
with open(ROM_FILE, 'w') as f:
    f.write(data)

### RAM test data generation

In [4]:
RAM_ADDR_WIDTH = 7
ADDR_COUNT = 2 ** RAM_ADDR_WIDTH
RAM_FILE = os.path.join(data_dir, 'RAM_test.mem')
data = '\n'.join([f'{i:02X}' for i in range(ADDR_COUNT)])
with open(RAM_FILE, 'w') as f:
    f.write(data)

In [5]:
def bin_format(num:int):
    return f'{num:02X}'

In [90]:
# ROM and RAM size
ROM_SIZE = 2**8
RAM_SIZE = 2**7

# ALU op-codes
ALU_OPS = Enum('ALU_OPS',
               names=['ADD', 'SUB', 'MUL', 'SL_A',
                      'SR_A', 'INC_A', 'INC_B', 'DEC_A',
                      'DEC_B', 'EQ', 'GT', 'LT',
                      'OUT_A'],
               start=0,
               type=int)

ALU_OPS_COMMENTS = {
    ALU_OPS.ADD : 'A + B',
    ALU_OPS.SUB : 'A - B', 
    ALU_OPS.MUL : 'A * B', 
    ALU_OPS.SL_A : 'A << 1', 
    ALU_OPS.SR_A : 'A >> 1', 
    ALU_OPS.INC_A : 'A + 1', 
    ALU_OPS.INC_B : 'B + 1', 
    ALU_OPS.DEC_A : 'A - 1', 
    ALU_OPS.DEC_B : 'B - 1', 
    ALU_OPS.EQ: 'A == B', 
    ALU_OPS.GT : 'A > B', 
    ALU_OPS.LT : 'A < B', 
    ALU_OPS.OUT_A : 'A'
}

# Instruction codes
READ_MEM_TO_A  = int('0000', base=2)
READ_MEM_TO_B  = int('0001', base=2)
WRITE_A_TO_MEM = int('0010', base=2)
WRITE_B_TO_MEM = int('0011', base=2)
ALU_OP_TO_A    = int('0100', base=2)
ALU_OP_TO_B    = int('0101', base=2)






def read_mem_to_A(mem_addr:int, ram_size=RAM_SIZE):
    assert mem_addr >= 0 and mem_addr < ram_size,\
            f'Memory address {mem_addr} out of range for {ram_size} bytes RAM!'
    
    comment = f' // A <- Mem[{mem_addr}]'
    
    return '\n'.join([bin_format(READ_MEM_TO_A) + comment, bin_format(mem_addr)])

def read_mem_to_B(mem_addr:int, ram_size=RAM_SIZE):
    assert mem_addr >= 0 and mem_addr < ram_size,\
            f'Memory address {mem_addr} out of range for {ram_size} bytes RAM!'
    
    comment = f' // B <- Mem[{mem_addr}]'
    
    return '\n'.join([bin_format(READ_MEM_TO_B) + comment, bin_format(mem_addr)])

def write_A_to_mem(mem_addr:int, ram_size=RAM_SIZE):
    assert mem_addr >= 0 and mem_addr < ram_size,\
            f'Memory address {mem_addr} out of range for {ram_size} bytes RAM!'
    
    comment = f' // Mem[{mem_addr}] <- A'
    
    return '\n'.join([bin_format(WRITE_A_TO_MEM) + comment, bin_format(mem_addr)])

def write_B_to_mem(mem_addr:int, ram_size=RAM_SIZE):
    assert mem_addr >= 0 and mem_addr < ram_size,\
            f'Memory address {mem_addr} out of range for {ram_size} bytes RAM!'
    
    comment = f' // Mem[{mem_addr}] <- B'
    
    return '\n'.join([bin_format(WRITE_B_TO_MEM) + comment, bin_format(mem_addr)])

def alu_to_A(op_code:int):
    assert op_code in ALU_OPS._value2member_map_, f'Op-code {op_code} is not valid!'
    
    comment = f' // A <- {ALU_OPS_COMMENTS[op_code]}'
    
    return bin_format((op_code << 4) + ALU_OP_TO_A) + comment

def alu_to_B(op_code:int):
    assert op_code in ALU_OPS._value2member_map_, f'Op-code {op_code} is not valid!'
    
    comment = f' // B <- {ALU_OPS_COMMENTS[op_code]}'
    
    return bin_format((op_code << 4) + ALU_OP_TO_B) + comment

In [88]:
def generate_rom(program, filename, size=2**8):
    rom = '\n'.join(program)
    rom_size = len(rom.split('\n'))
    assert rom_size <= size, f'Program is {rom_size} bytes for {size} bytes ROM.'
    
    # Fill rest of ROM with 00
    if rom_size < size:
        rom = rom + '\n' + '\n'.join(['FF']*(size-rom_size))
    
    with open(filename, 'w') as f:
        f.write(rom)
        
def generate_ram(data_entries, filename, size=2**7):
    ram = np.zeros(size, dtype=int)
    for i, d in data_entries:
        ram[i] = d
    
    ram = '\n'.join(map(bin_format, ram))
    with open(filename, 'w') as f:
        f.write(ram)

In [94]:
program = [
    # Memory instruction testing
    read_mem_to_A(0),
    read_mem_to_B(1),
    write_A_to_mem(2),
    write_B_to_mem(3),
    read_mem_to_A(3),
    read_mem_to_B(2),
    
    # ALU operation testing
    alu_to_A(ALU_OPS.ADD),
    alu_to_B(ALU_OPS.ADD),
    alu_to_B(ALU_OPS.SUB),
    alu_to_A(ALU_OPS.SUB),
    alu_to_A(ALU_OPS.MUL),
    alu_to_B(ALU_OPS.MUL),
    alu_to_A(ALU_OPS.SL_A),
    alu_to_A(ALU_OPS.SR_A),
    alu_to_B(ALU_OPS.SL_A),
    alu_to_B(ALU_OPS.SR_A),
    alu_to_A(ALU_OPS.INC_A),
    alu_to_A(ALU_OPS.DEC_A),
    alu_to_B(ALU_OPS.INC_B),
    alu_to_B(ALU_OPS.DEC_B),
    alu_to_A(ALU_OPS.EQ),
    alu_to_B(ALU_OPS.EQ),
    alu_to_A(ALU_OPS.EQ),
    alu_to_A(ALU_OPS.GT),
    alu_to_A(ALU_OPS.LT),
    alu_to_B(ALU_OPS.EQ),
    alu_to_B(ALU_OPS.LT),
    alu_to_B(ALU_OPS.GT),
    alu_to_B(ALU_OPS.INC_B),
    alu_to_B(ALU_OPS.OUT_A),
    alu_to_A(ALU_OPS.OUT_A)
]
rom_path = os.path.join(data_dir, 'ROM_processor_test.mem')
generate_rom(program, rom_path)

In [48]:
data_entries = [
    (0, 42),
    (1, 69)
]

ram_path = os.path.join(data_dir, 'RAM_processor_test.mem')
generate_ram(data_entries, ram_path)